In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import autograd
from torch.utils.data import DataLoader

import gc
import numpy as np

from utils.dataset import DetectionFolder
from utils.model import YoloV3, YoloLoss


In [ ]:
# config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float

data_config = { }
data_config['train'] = '../data/detect/train.txt'
data_config['test'] = '../data/detect/train.txt'
data_config['image'] = '../data/detect/images/'
data_config['label'] = '../data/detect/labels/'

model_config = { }
model_config['device'] = device
model_config['size'] = (608, 608)
model_config['channel'] = 3
model_config['dtype'] = torch.float
model_config['anchors'] = [(10, 13), (16, 30), (33, 23), (30, 61), (62, 45), (59, 119), (116, 90), (156, 198), (373, 326)]
model_config['attribs'] = 5
model_config['debug_level'] = 1

train_config = { }
train_config['coef_noobj'] = 0.2
train_config['coef_coord'] = 20 / (608 * 608)
train_config['coef_total'] = 2
train_config['device'] = device
train_config['debug_level'] = 0
train_config['iou_threshold'] = 0.75


trainset = DetectionFolder(data_config['train'], data_config['image'], data_config['label'])
trainloader = DataLoader(trainset, batch_size = 2, num_workers = 4)
#testset = DetectionFolder(config['test'], config['image'], config['label'])
#testloader = DataLoader(testset)

In [ ]:
print(trainset.__len__())
for indx in range(0, 4):
    print(trainset.__getitem__(indx)['label'].shape)
    print(trainset.__getitem__(indx))
    print(trainset.__getitem__(0)['image'])

In [ ]:
for idx, batches in enumerate(trainloader):
    print(idx)
    print(batches['image'].shape)

In [ ]:
# create model
model = YoloV3(model_config)
model.to(model_config['device'])

In [ ]:
# create components
learning_rate = 0.1
loss_cache = []
#lr_func = lambda epoch: learning_rate * (0.97 ** epoch) if epoch > 30 else 0.01
lr_func = lambda epoch: learning_rate

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_func, last_epoch = -1)
loss_func = YoloLoss(train_config)

In [ ]:
def train_step(config, model, trainloader, loss_func, optimizer):
    model.train()
        
    global learning_rate
    global loss_cache
    
    avg_loss = []
    
    for idx, batches in enumerate(trainloader):
        if config['debug_level'] >= 2:
            print('index', idx)
            
        image = batches['image'].to(config['device'], dtype = config['dtype'])
        labels = batches['label'].to(config['device'], dtype = config['dtype'])# / config['size'][0]
        label_len = batches['label_len'].to(config['device'], dtype = torch.long)
        if config['debug_level'] >= 3:
            print('label shape : ', labels.shape)
        
        # forward
        out1, out2, out3 = model(image)
        
        if config['debug_level'] >= 2:
            print('out1.shape : ', out1.shape)
            print('out1[0] : ', out1[0])
            print('out2.shape : ', out2.shape)
            print('out2[0] : ', out2[0])
            print('out3.shape : ', out3.shape)
            print('out3[0] : ', out3[0])
            print('labels.shape : ', labels.shape)
            print('labels[0] : ', labels[0])

        # clear optimizer
        optimizer.zero_grad()
        
        # backward
        loss = loss_func(torch.cat((out1, out2, out3), 1), labels, label_len)
        loss.backward()
        optimizer.step()
            
        avg_loss.append(loss.item())
        
        # cleanup
        del image
        del labels
        del out1
        del out2
        del out3
        gc.collect()
        torch.cuda.empty_cache()
    
    # update learning_rate
    loss_cache.append(np.mean(avg_loss))
        
    if len(loss_cache) >= 10 and np.mean(loss_cache) < np.mean(loss_cache[-2:]) :
        print('decrease learning rate from : ', learning_rate)
        print('average of previous loss : ', np.mean(loss_cache))
        print('length of previous loss : ', len(loss_cache))
        learning_rate = learning_rate * 0.90
        loss_cache = []
        print('decrease learning rate to : ', learning_rate)
    if len(loss_cache) > 20 :
        loss_cache = loss_cache[15:]
        
    # print loss
    if config['debug_level'] >= 1:
        print('avg loss : ', np.mean(avg_loss))



In [ ]:
# single step
with torch.autograd.set_detect_anomaly(False):
    train_step(model_config, model, trainloader, loss_func, optimizer)
    scheduler.step()

In [ ]:
# value checking
for epoch in range(0, 30):
    print('epoch : ', epoch)
    with torch.autograd.set_detect_anomaly(False):
        trainset.shuffle()
        train_step(model_config, model, trainloader, loss_func, optimizer)
        scheduler.step()


In [ ]:

train_config['debug_level'] = 0
loss_func = YoloLoss(train_config)

for epoch in range(30, 1200):
    print('epoch : ', epoch)
    trainset.shuffle()
    train_step(model_config, model, trainloader, loss_func, optimizer)
    scheduler.step()
    
    if epoch % 200 == 0 :
        torch.save(model, './epoch_' + str(epoch) + '.dat')

In [ ]:

torch.save(model, './epoch_2000.dat')